# Resumen clase anterior


<img src="https://raw.githubusercontent.com/modcommet/Clases/master/metanogenesis.png" width="35%" align="left">
<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/clase4_matriz.jpg" width="55%" align="rigth"/>

Calcular el valor de los flujos metabólicos usando numpy.

1. Asumiendo que el flujo limitante es E1=10 [mmol/gDW/h] y que el organismo maximiza la producción de CH4.
2. Igual que 1 pero maximizando la producción de F420.

In [1]:
import numpy as np
from scipy.optimize import linprog
from numpy.linalg import solve

# E1,E2,E3,E4,E5,R1,R2,R3

A = np.array([
[1,0,0,0,0,-1,0,0],
[0,1,0,0,0,-1,-1,0],
[0,0,0,0,0,1,-1,1],
[0,0,0,-2,0,-1,1,1],
[0,0,-1,0,0,0,1,-1],
[0,0,0,0,-1,0,0,1]])        

b = np.array([0,0,0,0,0,0])
c = np.array([0,0,1,0,0,0,0,0])

LB=[10,0,0,0,0,0,0,0]       # [0,0,0,0,0,0]
UB=[10,1000000,1000000,1000000,1000000,1000000,1000000,1000000]     # ya sabes la idea

res = linprog(-c, A_eq=A, b_eq=b, bounds=list(zip(LB,UB)))
print('Optimal value:', res.fun, '\nX:', res.x)


Optimal value: -10.000000001210326 
X: [1.00000000e+01 5.00013871e+05 1.00000000e+01 4.99993871e+05
 4.99993871e+05 1.00000000e+01 5.00003871e+05 4.99993871e+05]


# Clase 6

Pasar lista

## Contenidos

* Resolver problemas de ingeniería metabólica a escala genómica
* Introducción a cobrapy



# Introducción a cobra


Para modelar el metabolismo celular a escala genómica utilizaremos la librería ```cobra```. La documentación oficial de cobra se encuentra disponible [aquí](https://cobrapy.readthedocs.io/en/latest/index.html). Hoy veremos los comandos escenciales de cobra y construiremos nuestra propia red metábolica.

## Instalación


Para instalar ```cobra``` debes abrir el terminal de anaconda (si usas Windowns, busca Anaconda Prompt ) y escribir en el siguiente comando:

```python
pip install cobra --user
```

Para probar que la librería se instlado correctamente escribe el siguiente comando en la próxima celda:

```python
import cobra
```

Si no hay mensajes de error, entonces ya tienes una versión funcional de cobra.

## Comandos básicos


Comezaremos cargado ```cobra``` y determinando que version está instalada (esto es útil para consultar la documentación adecuada; diferentes versiones de cobra tienen diferentes documentaciones)

In [2]:
import cobra
cobra.__version__

'0.22.1'

## Redes metabólicas a escala genómica


En Inglés Genome-scale metabolic networks son redes metabólicas que incluyen todas las reacciones metabólicas presentes en el genoma del organismo bajo estudio.

<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/clase4_eColiMetabolis.png" width="45%" align="left">

En el caso de *Escherichia coli* la red consiste en  reacciones y metabolitos.

["Fuente KEGG"](https://www.genome.jp/kegg-bin/show_pathway?eco01100+M00125)

In [3]:
import cobra.test

# "ecoli" and "salmonella" are also valid arguments
model = cobra.test.create_test_model("ecoli")

Veamos cuantas reacciones y metabolitos tiene "ecoli"

In [4]:
print( "Número de reacciones: " , len(model.reactions) )
print( "Número de metabolitos: ", len(model.metabolites) )

Número de reacciones:  2583
Número de metabolitos:  1805


Podemos inspeccionar reacciones individualmente, por ejemplo la reacción 30:

In [5]:
print( model.reactions[20].reaction )

3amp_e --> 


Ver las primeras 10 reacciones:

In [6]:
for i in range(10):
    print( model.reactions[i] )

DM_4crsol_c: 4crsol_c --> 
DM_5drib_c: 5drib_c --> 
DM_aacald_c: aacald_c --> 
DM_amob_c: amob_c --> 
DM_mththf_c: mththf_c --> 
DM_oxam_c: oxam_c --> 
BIOMASS_Ec_iJO1366_WT_53p95M: 0.000223 10fthf_c + 0.000223 2dmmql8_c + 2.5e-05 2fe2s_c + 0.000248 4fe4s_c + 0.000223 5mthf_c + 0.000279 accoa_c + 0.000223 adocbl_c + 0.499149 ala__L_c + 0.000223 amet_c + 0.28742 arg__L_c + 0.234232 asn__L_c + 0.234232 asp__L_c + 54.119975 atp_c + 0.000116 bmocogdp_c + 2e-06 btn_c + 0.004952 ca2_c + 0.000223 chor_c + 0.004952 cl_c + 0.002944 clpn160_p + 0.00229 clpn161_p + 0.00118 clpn181_p + 0.000168 coa_c + 2.4e-05 cobalt2_c + 0.008151 colipa_e + 0.129799 ctp_c + 0.000674 cu2_c + 0.088988 cys__L_c + 0.024805 datp_c + 0.025612 dctp_c + 0.025612 dgtp_c + 0.024805 dttp_c + 0.000223 enter_c + 0.000223 fad_c + 0.006388 fe2_c + 0.007428 fe3_c + 0.255712 gln__L_c + 0.255712 glu__L_c + 0.595297 gly_c + 0.154187 glycogen_c + 0.000223 gthrd_c + 0.209121 gtp_c + 48.752916 h2o_c + 0.000223 hemeO_c + 0.092056 his__

## Nota

¿Cuál es tu interpretación de la reacción número 7?

También podemos identificar una reacción por su nombre:

In [7]:
pgi = model.reactions.get_by_id("PGI")
print( pgi )
print( pgi.reaction )
print( pgi.name )


print(pgi.lower_bound, "< pgi <", pgi.upper_bound)

udcpdp = model.metabolites.get_by_id("udcpdp_c")
print( udcpdp.reactions )
print( udcpdp.formula )

PGI: g6p_c <=> f6p_c
g6p_c <=> f6p_c
Glucose-6-phosphate isomerase
-1000.0 < pgi < 1000.0
frozenset({<Reaction MPTG2 at 0x7f00a1b8ad30>, <Reaction UDCPDPS at 0x7f00a166b340>, <Reaction BIOMASS_Ec_iJO1366_WT_53p95M at 0x7f00a27fe850>, <Reaction MPTG at 0x7f00a1b8a760>, <Reaction UDCPDP at 0x7f00a166be80>, <Reaction BIOMASS_Ec_iJO1366_core_53p95M at 0x7f00a27fe490>})
C55H89O7P2


Podemos verificar el balance de masa de una reacción:

In [8]:
print(  "Balance de masa original" )
print(  pgi.reaction )
print(  pgi.check_mass_balance() )

print(  "Veamos que pasa con el balance de masa si ahora agregamos un metabolito a la reacción" )
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): -1})
print(  pgi.reaction )
print(  pgi.check_mass_balance() )

Balance de masa original
g6p_c <=> f6p_c
{}
Veamos que pasa con el balance de masa si ahora agregamos un metabolito a la reacción
g6p_c + h_c <=> f6p_c
{'charge': -1.0, 'H': -1.0}


In [9]:
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): 2})
print( pgi.reaction )
print( pgi.check_mass_balance() )

g6p_c <=> f6p_c + h_c
{'charge': 1.0, 'H': 1.0}


In [10]:
print( pgi.check_mass_balance() )


{'charge': 1.0, 'H': 1.0}


Tambien podemos ver cuales son las reacciones en las que un determinado metabólito está involucrado:

In [11]:
for rxn in model.metabolites.get_by_id("atp_c").reactions:
    print( rxn )

GLCabcpp: atp_c + glc__D_p + h2o_c --> adp_c + glc__D_c + h_c + pi_c
SADT2: atp_c + gtp_c + h2o_c + so4_c --> aps_c + gdp_c + pi_c + ppi_c
PA180abcpp: atp_c + h2o_c + pa180_c --> adp_c + h_c + pa180_p + pi_c
AACPS9: ACP_c + atp_c + octa_c --> amp_c + ocACP_c + ppi_c
ATPS4rpp: adp_c + 4.0 h_p + pi_c <=> atp_c + h2o_c + 3.0 h_c
NADK: atp_c + nad_c --> adp_c + h_c + nadp_c
CDPMEK: 4c2me_c + atp_c --> 2p4c2me_c + adp_c + h_c
COLIPAPabctex: atp_c + colipap_p + h2o_c --> adp_c + colipap_e + h_c + pi_c
CRNabcpp: atp_c + crn_p + h2o_c --> adp_c + crn_c + h_c + pi_c
4PEPTabcpp: LalaDgluMdapDala_p + atp_c + h2o_c --> LalaDgluMdapDala_c + adp_c + h_c + pi_c
SERASr: atp_c + h_c + ser__L_c <=> ppi_c + seramp_c
S2FE2SR: 2fe1s_c + atp_c + h2o_c + sufbcd_c + sufsesh_c --> adp_c + 5.0 h_c + pi_c + sufbcd_2fe2s_c + sufse_c
ACGAMK: acgam_c + atp_c --> acgam6p_c + adp_c + h_c
METTRS: atp_c + met__L_c + trnamet_c --> amp_c + mettrna_c + ppi_c
SHKK: atp_c + skm_c --> adp_c + h_c + skm5p_c
FCLK: atp_c + fcl_

# Simulación de flujos metabólicos

Para simular los flujos metabólicos utilizando FBA (optimizamos la velocidad de producción de biomasa:

In [12]:
solution=model.optimize()
print( model.summary() )
print( model.objective )

Objective
1.0 BIOMASS_Ec_iJO1366_core_53p95M = 0.9761293262947259

Uptake
------
Metabolite     Reaction      Flux  C-Number  C-Flux
     ca2_e     EX_ca2_e  0.005081         0   0.00%
      cl_e      EX_cl_e  0.005081         0   0.00%
 cobalt2_e EX_cobalt2_e  2.44E-05         0   0.00%
     cu2_e     EX_cu2_e 0.0006921         0   0.00%
     fe2_e     EX_fe2_e   0.01568         0   0.00%
  glc__D_e  EX_glc__D_e        10         6 100.00%
       k_e       EX_k_e    0.1905         0   0.00%
     mg2_e     EX_mg2_e  0.008468         0   0.00%
     mn2_e     EX_mn2_e 0.0006745         0   0.00%
    mobd_e    EX_mobd_e 0.0001259         0   0.00%
     nh4_e     EX_nh4_e     10.54         0   0.00%
     ni2_e     EX_ni2_e 0.0003153         0   0.00%
      o2_e      EX_o2_e     17.85         0   0.00%
      pi_e      EX_pi_e    0.9416         0   0.00%
     so4_e     EX_so4_e    0.2462         0   0.00%
     zn2_e     EX_zn2_e 0.0003329         0   0.00%

Secretion
---------
Metabolite    

In [13]:
print( model.objective )

Maximize
1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1


In [14]:
model.objective=model.reactions.get_by_id("XYLabcpp")

In [15]:
print( model.objective )

Maximize
1.0*XYLabcpp - 1.0*XYLabcpp_reverse_35686


## Visualización de los resultados


Para visualizar los resultados usaremos la herramienta [Escher](https://escher.readthedocs.io/en/latest/index.html), la cual puedes instalar siguiendo estas [instrucciones](https://anaconda.org/BjornFJohansson/escher)


Para instalar ```escher``` debes abrir el terminal de anaconda (si usas Windowns, busca Anaconda Prompt ) y escribir en el siguiente comando:

```python
pip install escher --user
```

In [24]:
import escher
from escher import Builder

In [25]:
# VISUALIZATION OF RESULTS
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
)

In [26]:
builder.reaction_data = solution.fluxes

In [27]:
builder

Builder(reaction_data={'DM_4crsol_c': 0.00021767683976372398, 'DM_5drib_c': 0.0002196290984163296, 'DM_aacald_…

## Ejercicio

Simula y los flujos metabólicos de *E. coli* cuando:
    
1. El flujo de consumo de glucosa ("EX_glc__D_e") es igual a -2 [mmol/gDW/h]
2. El limite inferior para el flujo de consumo de oxigeno ("EX_o2_e") es igual a -10 [mmol/gDW/h]

En cada caso ilustra la distribución de flujos usando ```escher```

In [20]:
model.reactions.get_by_id("EX_glc__D_e").bounds

(-10.0, 1000.0)

In [21]:
model.reactions.get_by_id("EX_glc__D_e").bounds=(-2,-2)
model.reactions.get_by_id("EX_glc__D_e").bounds

(-2, -2)

In [22]:
solution2=model.optimize()

In [23]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,2,6,100.00%
o2_e,EX_o2_e,1.294,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-0.3209,1,2.67%
glcn_e,EX_glcn_e,-1.947,6,97.33%
h2o_e,EX_h2o_e,-0.3209,0,0.00%
h_e,EX_h_e,-1.947,0,0.00%
